# Registering Users in Syft Datasite Server

By default users are not allowed to create a new account on the Syft Datasite Server. This notebook is a tutorial for Data Owners to enable guest signups on their deployments.

### Import packages

In [ ]:
SYFT_VERSION = ">=0.9,<1.0.0"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -q

In [ ]:
# syft absolute
import syft as sy

sy.requires(SYFT_VERSION)

### Launch a Syft Datasite Server

In [ ]:
server = sy.orchestra.launch(name="test-datasite-1", reset=True)

In [ ]:
# log into the server with default root credentials
root_client = server.login(email="info@openmined.org", password="changethis")

#### By default registration is disabled. Only `root_client` can register

In [ ]:
# The assumed state of this test is a server with signup set to False
# however if the tox task has set it to True you need to overwrite the setting
# before running the tests
# root_client.settings.allow_guest_signup(enable=False)

In [ ]:
# Register a new user using root credentials
response_1 = root_client.register(
    email="joker@gotham.com",
    password="joker123",
    password_verify="joker123",
    name="Joker",
)
response_1

In [ ]:
with sy.raises(sy.SyftException, show=True):
    server.register(
        email="batman@gotham.com",
        password="1rIzHAx6uQaP",
        password_verify="1rIzHAx6uQaP",
        name="Batman",
    )

In [ ]:
# Register a new user as a GUEST
with sy.raises(sy.SyftException, show=True):
    server.register(
        email="batman@gotham.com",
        password="1rIzHAx6uQaP",
        password_verify="1rIzHAx6uQaP",
        name="Batman",
    )

In [ ]:
# Register a new user as a GUEST
with sy.raises(sy.SyftException, show=True):
    server.register(
        email="robin@gotham.com",
        password="5v1ei4OM2N4m",
        password_verify="5v1ei4OM2N4m",
        name="Robin",
    )

In [ ]:
assert root_client.settings.get().signup_enabled is False

#### Now, if root user enable registration, then the guest clients can also register

In [ ]:
# Get the current settings of the server
root_client.settings.get()

In [ ]:
# Enable guest signups
root_client.settings.allow_guest_signup(enable=True)

In [ ]:
# Refresh the root client to fetch the updated settings
root_client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
# Register a new user as a GUEST
response_2 = server.register(
    email="batman@gotham.com",
    password="1rIzHAx6uQaP",
    password_verify="1rIzHAx6uQaP",
    name="Batman",
)
response_2

In [ ]:
# Register a new user as a GUEST
response_3 = server.register(
    email="robin@gotham.com",
    password="5v1ei4OM2N4m",
    password_verify="5v1ei4OM2N4m",
    name="Robin",
)
response_3

In [ ]:
assert root_client.settings.get().signup_enabled is True
assert isinstance(response_1, sy.SyftSuccess)
assert isinstance(response_2, sy.SyftSuccess)
assert isinstance(response_3, sy.SyftSuccess)

### Toggle signup again

In [ ]:
# Refresh the root client
root_client.settings.allow_guest_signup(enable=False)

In [ ]:
# Refresh the root client to fetch the updated settings
root_client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
# Register a new user as a GUEST
with sy.raises(sy.SyftException, show=True):
    server.register(
        email="bane@gotham.com",
        password="SKY5cC2zQPRP",
        password_verify="SKY5cC2zQPRP",
        name="Bane",
    )

In [ ]:
# Register a new user as a GUEST
with sy.raises(sy.SyftException, show=True):
    server.register(
        email="riddler@gotham.com",
        password="7eVGUuNDyH8P",
        password_verify="7eVGUuNDyH8P",
        name="Riddler",
    )

In [ ]:
assert root_client.settings.get().signup_enabled is False

In [ ]:
# Cleanup local datasite server

if server.server_type.value == "python":
    server.land()